In [142]:
import time
t=time.time()
import pandas as pd
import numpy as np
import util
source='data/source/'
def get_tss(mane,w=5000):
    cols=['#chrom','chromStart','chromEnd','strand','geneSymbol']
    tss=pd.read_csv(mane_,sep='\t',usecols=cols).drop_duplicates()
    tss=tss[~tss['#chrom'].isin(['chrX','chrY'])]
    tss['ts']=tss.apply(lambda x:x['chromStart'] if x['strand']=='+' else x['chromEnd'],axis=1)
    tss['tss']=tss['#chrom'].str[3:5].astype(int)*10**9+tss['ts']
    tss=tss.drop_duplicates('tss',keep=False)
    tss=tss.drop_duplicates('geneSymbol',keep=False)
    tss.index=tss['geneSymbol']
    tss['chrom']=tss['#chrom'].str[3:5].astype(int)
    tss=tss[['chrom','ts','tss']]
    tss['chromStart']=tss['tss'].astype(str).str[-9:].astype(int)-w
    tss['chromEnd']=tss['tss'].astype(str).str[-9:].astype(int)+w
    return tss
def get_h(tss,h1,h9):
    h1=util.bw2df(h1,tss)
    h9=util.bw2df(h9,tss)
    h=pd.concat([h1['beta'],h9['beta']],axis=1)
    h.columns=['h1','h9']
    h.index=h.index.rename('_')
    return h
def get_tss_distance(ind):
    ind['g']=ind.index.str.split('_').str[0].astype(int)*10**9+ind.index.str.split('_').str[1].astype(int)
    tssl=tss['tss'].sort_values().tolist()
    ind['tss']=ind.g.apply(lambda x:util.take_closest(tssl, x))
    ind['d']=ind.g-ind.tss
    tss['gene']=tss.index
    dic=tss[['tss','gene']].set_index('tss')['gene'].to_dict()
    ind['gene']=ind.tss.apply(lambda x: dic[x])  
    return ind
def get_ez(ezp_):
    ezp=pd.read_csv(ezp_,sep='\t',header=None,usecols=[0,1,2,4]).drop_duplicates()
    ezp=ezp[~ezp[0].isin(['chrX','chrY','chrM',np.nan])]
    ezp=ezp[~ezp[0].str.contains('Un')][~ezp[0].str.contains('random')]
    ezp['b']=ezp[0].str[3:5].astype(int)*10**9+ezp[1]
    ezp['e']=ezp[0].str[3:5].astype(int)*10**9+ezp[2]
    ezp['r']=ezp.apply(lambda x:range(x['b'],x['e']),axis=1)
    ezp=ezp[['r',4]]
    ezs=ezp.r.values
    ss=ezp[4].values
    ezd={}
    for i in range(len(ezs)):
        for e in ezs[i]:
            if e in ezd:
                ezd[e]=np.max([ezd[e],ss[i]])
            else:
                ezd[e]=ss[i]
    return ezd
def get_exp(ind_gene,tpm):
    dic={'Whole Blood':'exp_blood',
         'Skin - Not Sun Exposed (Suprapubic)':'exp_skin_se',
         'Skin - Sun Exposed (Lower leg)':'exp_skin_sp',
         'Cells - Cultured fibroblasts':'exp_fib',
        }
    tpm=pd.read_csv(tpm,sep='\t',skiprows=2,index_col=0)
    tpm=tpm.drop_duplicates('Description',keep=False)
    tpm.index=tpm['Description']
    tpm=tpm[list(dic.keys())].rename(dic,axis=1)
    exp=ind_gene.merge(tpm,left_on='gene',right_index=True,how='left')
    return exp

In [30]:
# get TSS
mane_=source+'mane.gz'
tss=get_tss(mane_)
tss.to_csv('data/tss.csv')

In [3]:
# get H
h1=source+'h/Human_H1.meth.bw'
h9=source+'h/Human_H9.meth.bw'
h=get_h(tss,h1,h9)
h.to_csv('data/h.csv')

1.2.2.3.5.6.7.8.9.11.12.13.15.16.17.19.20.22.(3623674, 1)
1.2.2.3.5.6.7.8.9.11.12.13.15.16.17.19.20.22.(3623674, 1)


In [ ]:
# get ind
ind=pd.read_csv('data/h.csv',index_col=0,usecols=['_'])
ind=get_tss_distance(ind)
ind.to_csv('data/ind.csv')

In [152]:
#ind=pd.read_csv('data/ind.csv',index_col=0)

In [143]:
# get EZ
ezp_=source+'ENCFF414CAB.bed.gz'
ez=get_ez(ezp_)
ind['ez']=ind.g.apply(lambda x:ez[x] if x in ez else 0)
ind[['ez']].to_csv('data/ez.csv')

In [11]:
# get exp
ind_gene=ind[['gene']]
tpm=source+'GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_median_tpm.gct.gz'
exp=get_exp(ind_gene,tpm)
exp.to_csv('data/exp.csv')

In [ ]:
#egt neo


In [ ]:
print(round(time.time()-t))